In [ ]:
# Celda 1 — Configuración e imports

from pathlib import Path
import numpy as np
import pandas as pd
import librosa

# Carpeta con los .wav (una subcarpeta por especie)
IN_DIR = "../data/Tinamidae"

# Archivo de salida
OUT_CSV = "../traits/output_csv/TC.csv"

# Audio
SR_TARGET = 44100  # Hz

# STFT (ventana Hann de 1024 muestras, 50% solape, escala lineal)
WIN_LENGTH   = 1024
HOP_LENGTH   = WIN_LENGTH // 2
STFT_CENTER  = False   # para que los frames sean consistentes con RMS

# Umbral de energía para excluir frames silenciosos
USE_ENERGY_THRESHOLD = True
ENERGY_DB_FLOOR      = -60.0  # dB relativos al máximo RMS del archivo


In [ ]:
# Celda 2 — Utilidades: I/O, z-score, STFT de potencia, RMS y TC por frame

def list_wavs(base_dir: str):
    base = Path(base_dir)
    return sorted([p for p in base.rglob("*.wav") if p.is_file()])

def species_from_path(p: Path, base_dir: str):
    """
    Asume estructura: base_dir/especie/archivo.wav
    Devuelve el nombre de la carpeta inmediatamente bajo base_dir.
    """
    base = Path(base_dir).resolve()
    rel  = p.resolve().relative_to(base)
    return rel.parts[0] if len(rel.parts) > 1 else "UNKNOWN"

def load_mono(fp: Path):
    """
    Carga audio mono a SR_TARGET.
    """
    y, sr = librosa.load(str(fp), sr=SR_TARGET, mono=True)
    return y.astype(np.float64, copy=False), SR_TARGET

def zscore_signal(y: np.ndarray, eps: float = 1e-12):
    """
    Estandariza la señal: (y - media) / sigma.
    TC es invariante a reescalados globales, pero así respetamos la definición
    basada en la señal estandarizada \hat{x}[n].
    """
    if y.size == 0:
        return y
    mu = float(np.mean(y))
    sd = float(np.std(y, ddof=0))
    return (y - mu) / (sd + eps)

def stft_power(y_hat: np.ndarray):
    """
    Espectrograma de POTENCIA S^2 = |STFT|^2 (frecuencia x tiempo)
    usando ventana Hann y escala lineal.
    """
    D = librosa.stft(
        y=y_hat,
        n_fft=WIN_LENGTH,
        hop_length=HOP_LENGTH,
        window="hann",
        center=STFT_CENTER,
    )
    S2 = np.abs(D) ** 2
    return S2

def frame_rms_db(y: np.ndarray):
    """
    Energía RMS por frame en dB, relativa al máximo RMS del archivo.
    Sirve para filtrar frames silenciosos (< -60 dB).
    """
    rms = librosa.feature.rms(
        y=y,
        frame_length=WIN_LENGTH,
        hop_length=HOP_LENGTH,
        center=STFT_CENTER,
    )[0]  # shape (T,)
    rms_db = librosa.amplitude_to_db(rms, ref=np.max)  # <= 0 dB
    return rms_db

def tc_series_from_S2(S2: np.ndarray):
    """
    A partir del espectrograma de POTENCIA S^2 (freq x time):
    1) Calcula SFM[t] (spectral flatness measure) por frame.
    2) Define TC_t = -log(SFM[t])  → alto = más tonal.
    Devuelve TC_t (1D, longitud n_frames).
    """
    sfm = librosa.feature.spectral_flatness(S=S2)[0]  # SFM[t] in [0,1]
    TC_t = -np.log(sfm)
    return TC_t


In [ ]:
# Celda 3 — Cálculo de \hat{TC} por archivo (mediana de TC_t en frames válidos)

files = list_wavs(IN_DIR)
print(f"Archivos .wav encontrados: {len(files)}")

rows = []

for fp in files:
    especie = species_from_path(fp, IN_DIR)
    try:
        # Carga y estandariza la señal
        y, sr = load_mono(fp)
        y_hat  = zscore_signal(y)

        # Espectrograma de potencia
        S2 = stft_power(y_hat)
        n_frames_total = int(S2.shape[1])

        # Máscara de frames válidos según energía RMS
        valid_mask = np.ones(n_frames_total, dtype=bool)
        if USE_ENERGY_THRESHOLD:
            rms_db = frame_rms_db(y)
            # alineamos longitudes por seguridad
            nT = min(n_frames_total, rms_db.shape[0])
            valid_mask[:] = False
            valid_mask[:nT] = rms_db[:nT] >= ENERGY_DB_FLOOR

        n_frames_valid = int(np.sum(valid_mask))

        if n_frames_valid == 0:
            # Sin frames válidos --> NaN
            rows.append({
                "species": especie,
                "relpath": str(fp),
                "sr": sr,
                "T_frames_total": n_frames_total,
                "T_frames_valid": 0,
                "TC_hat_file": np.nan,
            })
            continue

        # Serie TC_t en frames válidos y mediana por archivo (TC_hat)
        TC_t = tc_series_from_S2(S2[:, valid_mask])
        TC_hat_file = float(np.median(TC_t))

        rows.append({
            "species": especie,
            "relpath": str(fp),
            "sr": sr,
            "T_frames_total": n_frames_total,
            "T_frames_valid": n_frames_valid,
            "TC_hat_file": TC_hat_file,   # \hat{TC} por archivo
        })

    except Exception as e:
        print(f"[WARN] Error en {fp}: {e}")
        rows.append({
            "species": especie,
            "relpath": str(fp),
            "sr": np.nan,
            "T_frames_total": np.nan,
            "T_frames_valid": 0,
            "TC_hat_file": np.nan,
            "error": f"{type(e).__name__}: {e}",
        })

df_tc_files = pd.DataFrame(rows)
print("Archivos procesados:", df_tc_files.shape[0])
df_tc_files.head()


In [ ]:
# Celda 4 — Resumen por especie y export de TC.csv

df_valid = df_tc_files.dropna(subset=["TC_hat_file"]).copy()

df_species = (
    df_valid
    .groupby("species")
    .agg(
        n_muestras          = ("relpath", "count"),
        T_frames_valid_mean = ("T_frames_valid", "mean"),
        TC_hat_mean         = ("TC_hat_file", "mean"),
        TC_hat_median       = ("TC_hat_file", "median"),
    )
    .reset_index()
    .sort_values("n_muestras", ascending=False)
)

print("Especies con datos:", df_species.shape[0])
df_species.head(10)

df_species.to_csv(OUT_CSV, index=False)
print(f"[OK] Archivo guardado en: {OUT_CSV}")
